# Prototype for the Financial Advisor Template - Interpretable Strategy Engine

In [9]:
TICKERS_GOOD = ['MCD', 'TMO']
TICKERS_BAD = ['COP', 'NKE']
ALL = TICKERS_GOOD + TICKERS_BAD

	## 1 Data ingestion & cleaning

In [8]:
!python ../src/ingestion/fetch_data.py --tickers {' '.join(ALL)} --start 2015-01-01 
!python ../src/ingestion/clean_data.py --tickers {' '.join(ALL)}

Saved data for AAPL to c:\FinalProject\src\ingestion\..\..\data\raw/AAPL.csv
Saved data for ABBV to c:\FinalProject\src\ingestion\..\..\data\raw/ABBV.csv
Saved data for ABT to c:\FinalProject\src\ingestion\..\..\data\raw/ABT.csv
Saved data for ACN to c:\FinalProject\src\ingestion\..\..\data\raw/ACN.csv
Saved data for ADBE to c:\FinalProject\src\ingestion\..\..\data\raw/ADBE.csv
Saved data for AIG to c:\FinalProject\src\ingestion\..\..\data\raw/AIG.csv
Saved data for AMD to c:\FinalProject\src\ingestion\..\..\data\raw/AMD.csv
Saved data for AMGN to c:\FinalProject\src\ingestion\..\..\data\raw/AMGN.csv
Saved data for AMT to c:\FinalProject\src\ingestion\..\..\data\raw/AMT.csv
Saved data for AMZN to c:\FinalProject\src\ingestion\..\..\data\raw/AMZN.csv
Saved data for AVGO to c:\FinalProject\src\ingestion\..\..\data\raw/AVGO.csv
Saved data for AXP to c:\FinalProject\src\ingestion\..\..\data\raw/AXP.csv
Saved data for BA to c:\FinalProject\src\ingestion\..\..\data\raw/BA.csv
Saved data for 


[                       0%                       ]
[*                      2%                       ]  2 of 105 completed
[*                      3%                       ]  3 of 105 completed
[**                     4%                       ]  4 of 105 completed
[**                     5%                       ]  5 of 105 completed
[***                    6%                       ]  6 of 105 completed
[***                    7%                       ]  7 of 105 completed
[***                    7%                       ]  7 of 105 completed
[****                   9%                       ]  9 of 105 completed
[*****                 10%                       ]  10 of 105 completed
[*****                 10%                       ]  11 of 105 completed
[*****                 11%                       ]  12 of 105 completed
[******                12%                       ]  13 of 105 completed
[******                13%                       ]  14 of 105 completed
[*******            

Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AAPL.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\ABBV.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\ABT.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\ACN.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\ADBE.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AIG.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AMD.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AMGN.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AMT.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AMZN.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AVGO.csv (2726 rows)
Cleaned → c:\FinalProject\src\ingestion\..\..\data\processed\AXP.csv (2726 rows)
Cleaned → c:\FinalProj

c:\FinalProject\src\ingestion\clean_data.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)                # propagate last good
c:\FinalProject\src\ingestion\clean_data.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)                # propagate last good
c:\FinalProject\src\ingestion\clean_data.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)                # propagate last good
c:\FinalProject\src\ingestion\clean_data.py:12: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill',

## 2 Feature engineering (18 TA factors)